<a href="https://colab.research.google.com/github/poomshift/PromptAlchemist_SD_Webui_Colab/blob/main/PromptAlchemist_SD_Webui_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title  { form-width: "30%" }
from IPython.display import Javascript

display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 500})'''))
from IPython.display import clear_output, display, HTML
import os
import time
from datetime import timedelta
from google.colab import drive
from IPython.utils import capture
from subprocess import getoutput
from urllib.parse import unquote
from google.colab.output import eval_js
import shutil

%cd /content
drive.mount('/content/drive')

%env TF_CPP_MIN_LOG_LEVEL=1

!apt -y install -qq aria2 libcairo2-dev pkg-config python3-dev
!pip install -q xformers==0.0.20 triton==2.0.0 -U
# @markdown เวอร์ชั่นของ WebUI
Version = "v1.6.0" # @param ["v1.6.0", "v1.5.2"] {type:"string"}
!git clone -b {Version} https://github.com/AUTOMATIC1111/stable-diffusion-webui
!git clone https://huggingface.co/embed/negative /content/stable-diffusion-webui/embeddings/negative
!git clone https://huggingface.co/embed/lora /content/stable-diffusion-webui/models/Lora/positive
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/embed/upscale/resolve/main/4x-UltraSharp.pth -d /content/stable-diffusion-webui/models/ESRGAN -o 4x-UltraSharp.pth
!git clone https://github.com/camenduru/tunnels /content/stable-diffusion-webui/extensions/tunnels
!git clone https://github.com/etherealxx/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
!git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete   /content/stable-diffusion-webui/extensions/a1111-sd-webui-tagcomplete
!git clone https://github.com/Coyote-A/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui/extensions/ultimateSD
!git clone https://github.com/richrobber2/canvas-zoom /content/stable-diffusion-webui/extensions/canvas-zoom
!git clone https://github.com/zanllp/sd-webui-infinite-image-browsing /content/stable-diffusion-webui/extensions/sd-webui-infinite-image-browsing

# @markdown ส่วนเสริม
Controlnet = True # @param {type:"boolean"}
if Controlnet == True:
  !git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
else:
  ()

Adetailer = False # @param {type:"boolean"}
if Adetailer == True:
  !git clone https://github.com/Bing-su/adetailer /content/stable-diffusion-webui/extensions/adetailer
else:
  ()

NSFW_Censor = True # @param {type:"boolean"}
if NSFW_Censor == True:
  !git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui-nsfw-censor /content/stable-diffusion-webui/extensions/stable-diffusion-webui-nsfw-censo
else:
  ()

ReActor = False # @param {type:"boolean"}
if ReActor == True:
  !git clone https://github.com/Gourieff/sd-webui-reactor /content/stable-diffusion-webui/extensions/sd-webui-reactor
  !pip install insightface==0.7.3
else:
  ()
StyleSelectorXL = False # @param {type:"boolean"}
if StyleSelectorXL == True:
  !git clone https://github.com/ahgsql/StyleSelectorXL.git /content/stable-diffusion-webui/extensions/StyleSelectorXL
  !pip install insightface==0.7.3
else:
  ()

%cd /content/stable-diffusion-webui

# Define the file path and content
file_path = '/content/stable-diffusion-webui/config.json'
config_content = '''
{
  "outdir_samples": "",
  "outdir_txt2img_samples": "/content/drive/MyDrive/SD-IMG-OUT/outputs/text",
  "outdir_img2img_samples": "/content/drive/MyDrive/SD-IMG-OUT/outputs/image",
  "outdir_extras_samples": "/content/drive/MyDrive/SD-IMG-OUT/outputs/extras",
  "outdir_grids": "",
  "outdir_txt2img_grids": "/content/drive/MyDrive/SD-IMG-OUT/outputs/grids",
  "outdir_img2img_grids": "/content/drive/MyDrive/SD-IMG-OUT/outputs/grids",
  "outdir_save": "/content/drive/MyDrive/SD-IMG-OUT/outputs/save",
  "outdir_init_images": "/content/drive/MyDrive/SD-IMG-OUT/outputs/init-images"
}
'''
# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Write the config file
with open(file_path, 'w') as config_file:
    config_file.write(config_content)

!git reset --hard
!git -C /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai reset --hard

%cd /content/stable-diffusion-webui

#controlnet model
if Controlnet == True:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1e_sd15_tile_fp16.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_ip2p_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11e_sd15_shuffle_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_canny_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1p_sd15_depth_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1p_sd15_depth_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_inpaint_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_lineart_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_mlsd_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_normalbae_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_openpose_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_scribble_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_scribble_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_seg_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_seg_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15_softedge_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11p_sd15s2_lineart_anime_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/raw/main/control_v11f1e_sd15_tile_fp16.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11f1e_sd15_tile_fp16.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v1p_sd15_qrcode_monster.safetensors
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.yaml -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v1p_sd15_qrcode_monster.yaml
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o ip-adapter_sd15.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o ip-adapter-plus-face_sd15.pth
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o ip-adapter-plus_sd15.pth
else:
  ()
# @markdown VAE
VAE = "https://civitai.com/api/download/models/87822" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {VAE} -d /content/stable-diffusion-webui/models/VAE
# @markdown Checkpoint
Checkpoint_01 = "https://civitai.com/api/download/models/148087" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {Checkpoint_01} -d /content/stable-diffusion-webui/models/Stable-diffusion
Checkpoint_02 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {Checkpoint_02} -d /content/stable-diffusion-webui/models/Stable-diffusion
Checkpoint_03 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {Checkpoint_03} -d /content/stable-diffusion-webui/models/Stable-diffusion
# @markdown LoRA
LoRA_01 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_01} -d /content/stable-diffusion-webui/models/Lora
LoRA_02 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_02} -d /content/stable-diffusion-webui/models/Lora
LoRA_03 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_03} -d /content/stable-diffusion-webui/models/Lora
LoRA_04 = "" #@param {type:"string"}
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {LoRA_04} -d /content/stable-diffusion-webui/models/Lora

%cp -av /content/drive/MyDrive/Lora /content/stable-diffusion-webui/models/Lora
%cp -av /content/drive/MyDrive/Checkpoint /content/stable-diffusion-webui/models/Stable-diffusion

!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared_options.py
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [17]:
rm -r /content/ui

In [ ]:
%cd /content/stable-diffusion-webui
!sed -i -e 's/\["sd_model_checkpoint"\]/\["sd_model_checkpoint","sd_vae","CLIP_stop_at_last_layers"\]/g' /content/stable-diffusion-webui/modules/shared_options.py
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple --autolaunch